# Parsing Tabular Data

**Originally Contributed by**: Clayton Barrows

## Introduction

An example of how to parse tabular files (CSV) files similar to the format established in
the [RTS-GMLC](github.com/gridmod/rts-gmlc/RTS_Data/SourceData) and create a `System` using
[PowerSystems.jl](github.com/NREL/PowerSystems.jl)

### Environment
This notebook depends on the SIIPExamples.jl environment which is loaded by default

In [1]:
using Pkg
Pkg.status()

Project SIIPExamples v0.0.1
    Status `~/Documents/repos/Examples/Project.toml`
  [9961bab8] Cbc v0.6.6
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.20.0
  [2cd47ed4] InfrastructureSystems v0.5.10
  [b6b21f68] Ipopt v0.6.1
  [2535ab7d] JSON2 v0.3.1
  [4076af6c] JuMP v0.21.1
  [98b081ad] Literate v2.2.1
  [91a5bcdd] Plots v0.29.2
  [5f7eddb3] PowerGraphics v0.1.0 #master (https://github.com/nrel-siip/powergraphics.jl)
  [e690365d] PowerSimulations v0.2.2
  [bcd98974] PowerSystems v0.11.2
  [9e3dc215] TimeSeries v0.16.0
  [ade2ca70] Dates 
  [56ddb016] Logging 
  [9a3f8284] Random 
  [10745b16] Statistics 
  [8dfed614] Test 


### Dependencies

In [2]:
using SIIPExamples
using PowerSystems
using TimeSeries
using Dates
const PSY = PowerSystems
const IS = PSY.InfrastructureSystems;

### Fetch Data
PowerSystems.jl links to some test data that is suitable for this example.
Let's download the test data

In [3]:
PSY.download(PSY.TestData; branch = "master", force=true)
base_dir = dirname(dirname(pathof(PowerSystems)));

### The tabular data format relies on a folder containing `*.csv` files and a `user_descriptors.yaml` file
First, we'll read the tabular data

In [4]:
RTS_GMLC_DIR = joinpath(base_dir,"data/RTS_GMLC");
rawsys = PSY.PowerSystemTableData(RTS_GMLC_DIR,100.0, joinpath(RTS_GMLC_DIR,"user_descriptors.yaml"))

┌ Info: Parsing csv files in Reserves ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:130
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:134
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:134
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:134
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:134
┌ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R1.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/pow

PowerSystems.PowerSystemTableData:
  directory:  /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/data/RTS_GMLC
  timeseries_metadata_file:  /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/data/RTS_GMLC/timeseries_pointers.json
  basepower:  100.0
  BRANCH:  120×14 DataFrame
  RESERVE:  7×7 DataFrame
  BUS:  73×15 DataFrame
  GENERATOR:  158×57 DataFrame
  STORAGE:  22×6 DataFrame
  DC_BRANCH:  1×60 DataFrame


### Create a `System`
Next, we'll create a `System` from the `rawsys` data. Since a `System` is predicated on a
forecast resolution and the `rawsys` data includes both 5-minute and 1-hour resolution
forecasts, we also need to specify which forecasts we want to include in the `Systeem`.
The `forecast_resolution` kwarg filters to only include forecsats with a matching resolution.

In [5]:
sys = System(rawsys; forecast_resolution = Dates.Hour(1));
sys

┌ Info: Adding contributing generators for Spin_Up_R1 by category
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:640
┌ Info: Adding contributing generators for Spin_Up_R2 by category
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:640
┌ Info: Adding contributing generators for Spin_Up_R3 by category
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:640
┌ Info: Adding contributing generators for Flex_Up by category
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:640
┌ Info: Adding contributing generators for Flex_Down by category
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/N15Uk/src/parsers/power_system_table_data.jl:640
┌ Info: Adding contributing generators for Reg_Up by category
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSys

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Area,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
2,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,73
3,GenericBattery,Storage <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
4,HVDCLine,DCBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
5,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
6,HydroEnergyReservoir,HydroGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,19
7,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,104
8,LoadZone,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,21
9,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,51


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*